In [1]:
import numpy as np
from coverage import gen_sats, gen_times, camera_model, get_inst_fov, get_inst_fov2, los_to_earth, get_lvlh_pointing
from datetime import datetime, timezone, timedelta
import dataclasses
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import branca
import folium

from skyfield.api import load, wgs84, Distance
from skyfield.framelib import itrs
from skyfield.toposlib import ITRSPosition
from landsat import Instrument, Platform, Scene

In [2]:
start_dt = datetime.fromisoformat(Scene.start_utc)
num_days = 2

tles = gen_sats(
    sat_nos=[Platform.norad_id] # How to best handle multiple platforms? (TLE vs. SPG4 model too)
    # sat_nos=[39084]
    # sat_nos=[39084,49260]
)

inst = camera_model(
    name=Instrument.name, 
    fl=Instrument.focal_length_mm, 
    pitch=Instrument.pitch_um*1e-3, 
    h_pix=Instrument.rows, 
    v_pix=Instrument.cols, 
)

times = gen_times(
    start_yr=start_dt.year,
    start_mo=start_dt.month, 
    start_day=start_dt.day, 
    days=num_days, 
    step_min=Instrument.img_period)

xcell_size = ycell_size = .1

Satellite(s) Loaded from TLE:

[<EarthSatellite LANDSAT 9 catalog #49260 epoch 2022-06-15 11:12:37 UTC>]

Propogation time: 
 2022-07-27 00:00:00+00:00 
to 
 2022-07-28 23:59:48+00:00

In [3]:
sat = tles[0][0]

In [4]:
# %%timeit
# get_inst_fov(sat, times[0], inst)

In [5]:
llas = get_inst_fov(sat, times[0], inst)

In [6]:
# %%timeit
# get_inst_fov2(sat, times[0], inst)

In [7]:
llas2 = get_inst_fov2(sat, times[0], inst)

In [8]:
llas2

{'c1': {'lat': -63.38163240859396, 'lon': -12.146297825585277},
 'c2': {'lat': -62.81058302898076, 'lon': -8.709924300718592},
 'c3': {'lat': -61.20749510204343, 'lon': -10.024883795288902},
 'c4': {'lat': -61.7480644487944, 'lon': -13.301729363872125}}

In [9]:
llas

{'c1': {'lat': -63.38163240859396, 'lon': -12.146297825585277},
 'c2': {'lat': -62.81058302898076, 'lon': -8.709924300718592},
 'c3': {'lat': -61.20749510204343, 'lon': -10.024883795288902},
 'c4': {'lat': -61.7480644487944, 'lon': -13.301729363872125}}

In [10]:
from scipy.spatial.transform import Rotation
from functools import partial

time = times[0]


cs_lla_dict = {
    "c1": {"lat": None, "lon": None},
    "c2": {"lat": None, "lon": None},
    "c3": {"lat": None, "lon": None},
    "c4": {"lat": None, "lon": None},
}


In [11]:
dx = inst['hfov_deg']/2
dy = inst['vfov_deg']/2

# dx = 0
# dy = 0

In [12]:
# %%timeit

# for time in times:

lvlh = get_lvlh_pointing(sat, time)
xyz_dist_rates = sat.at(time).frame_xyz_and_velocity(itrs)
xyz_dist = xyz_dist_rates[0]
z_rate = xyz_dist_rates[1]

lvlh_mat = np.array(list(get_lvlh_pointing(sat, time).values()))
lvlh_rot = Rotation.from_matrix(lvlh_mat)

corners = lvlh_rot.from_euler('xy', [[-dx,dy],[dx,dy],[dx,-dy],[-dx,-dy]], degrees=True)

los_XY = corners.apply(lvlh["Z"])

_los_to_earth = partial(los_to_earth, xyz_dist.km)
los_xyz = np.apply_along_axis(_los_to_earth, 1, los_XY)

c = 0
keys = list(cs_lla_dict.keys())

for xyz in los_xyz:
    key = keys[c]
    los_itrs = ITRSPosition(Distance(km=xyz))
    los_lat, los_lon = wgs84.latlon_of(los_itrs.at(time))
    cs_lla_dict[key]["lat"] = los_lat.degrees
    cs_lla_dict[key]["lon"] = los_lon.degrees
    c += 1

In [13]:
cs_lla_dict

{'c1': {'lat': -61.64043505606069, 'lon': -9.228009639599216},
 'c2': {'lat': -61.3102216808999, 'lon': -12.248632272568662},
 'c3': {'lat': -62.99211902391925, 'lon': -12.959494967930768},
 'c4': {'lat': -63.302988761767196, 'lon': -9.751259001664163}}

In [14]:
llas

{'c1': {'lat': -63.38163240859396, 'lon': -12.146297825585277},
 'c2': {'lat': -62.81058302898076, 'lon': -8.709924300718592},
 'c3': {'lat': -61.20749510204343, 'lon': -10.024883795288902},
 'c4': {'lat': -61.7480644487944, 'lon': -13.301729363872125}}